In [ ]:
!pip uninstall -y tensorboard
!pip uninstall -y tensorflow
!pip uninstall -y autoai-libs

!pip install -U geocoder
!pip install -U geopy
!pip install -U googlemaps
!pip install -U folium
!pip install -U numpy
!pip install -U pandas

import pandas as pd
import numpy as np

from pandas import json_normalize  # tranform JSON file into a pandas dataframe

import googlemaps
from datetime import datetime
import requests
import geopy

geopy.geocoders.options.default_user_agent = "my-application"

import ast
import json
import time

import folium
from folium.plugins import HeatMap

from ibm_botocore.client import Config
import ibm_boto3

print('Ready')

In [ ]:
# The code was removed by Watson Studio for sharing.

In [ ]:
!wget --cookies=on --save-cookies cookies.txt --keep-session-cookies --post-data 'user=EMAIL&password=PASSWORD' 'https://www.ctt.pt/fecas/login?service=https%3A%2F%2Fwww.ctt.pt%2Ffemgu%2Fj_spring_cas_security_check.jspx'
!wget --cookies=on --load-cookies cookies.txt --keep-session-cookies 'https://www.ctt.pt/feapl_2/app/restricted/postalCodeSearch/postalCodeDownloadFiles!downloadPostalCodeFile.jspx' -O PostCodeFull.Pt.zip

In [ ]:
# The code was removed by Watson Studio for sharing.

In [ ]:
def download_file_cos(credentials,local_file_name,key):  
    cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])
    try:
        res=cos.download_file(Bucket=credentials['BUCKET'],Key=key,Filename=local_file_name)
    except Exception as e:
        print(Exception, e)
    else:
        print('File Downloaded')

In [ ]:
def upload_file_cos(credentials,local_file_name,key):  
    cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])
    try:
        res=cos.upload_file(Filename=local_file_name, Bucket=credentials['BUCKET'],Key=key)
    except Exception as e:
        print(Exception, e)
    else:
        print(' File Uploaded')

In [ ]:
!ls -la

In [ ]:
# This ZIP contains (distritos.txt, concelhos.txt, todos_cp.txt)
!unzip -x PostCodeFull.Pt.zip

In [ ]:
df_DD=pd.read_csv('distritos.txt',sep=';',encoding='iso8859_15')
df_DD.head()

In [ ]:
df_CC=pd.read_csv('concelhos.txt',sep=';',encoding='iso8859_15')
df_CC=df_CC[df_CC['DD']==8]
df_CC.head()

In [ ]:
df_PostalCode=pd.read_csv('todos_cp.txt',sep=';',encoding='iso8859_15', index_col=False).reset_index()[['DD','CC','LLLL','LOCALIDADE','CP4','CP3','CPALF']]
df_PostalCode.head()

In [ ]:
# Select only location from Algarve
df_CPALF_Algarve=pd.DataFrame(df_PostalCode[df_PostalCode['DD']==8].groupby(['CPALF'])['CP4'].count()).reset_index()[['CPALF']]
df_CPALF_Algarve.head()

In [ ]:
# The code was removed by Watson Studio for sharing.

In [ ]:
df_all_locations=pd.DataFrame()
for ind in df_CPALF_Algarve.index:
    address='{}, Algarve, Portugal'.format(df_CPALF_Algarve['CPALF'][ind])
    try:
        gmaps = googlemaps.Client(key=GOOGLE_MAPS_API_KEY,timeout=1)
        location = gmaps.geocode(address)
    except geopy.exc.GeocoderUnavailable:
        print('GeocoderUnavailable')
        print(name)
    except urllib.error.URLError:
        print('URLError')
        print(name)
    except OSError:
        print('OSError')
        print(name)


    if df_all_locations.empty:
        df_all_locations=pd.DataFrame(json_normalize(location))
    else:
        df_all_locations=df_all_locations.append(json_normalize(location),sort=False)

print(df_all_locations.shape)

In [ ]:
df_location_geometry=df_all_locations.reset_index()[['geometry.location.lat','geometry.location.lng']].reset_index()
df_location_geometry.to_csv('location_geometry.csv',index=False)
upload_file_cos(credentials,'location_geometry.csv','location_geometry.csv')

In [ ]:
# Remove duplications
df_location_geometry_sorted=df_location_geometry.sort_values(by=['geometry.location.lat', 'geometry.location.lng'],ignore_index=True).reset_index()[['geometry.location.lat','geometry.location.lng']]
df_location_geometry_sorted.head()

In [ ]:
# The code was removed by Watson Studio for sharing.

In [ ]:
#category='4bf58dd8d48988d12f951735' # Resort
category='4bf58dd8d48988d1fa931735' # Hotel

df_all_hotels=pd.DataFrame()
for ind in df_location_geometry_sorted.index:

    latitude = df_location_geometry_sorted['geometry.location.lat'][ind]
    longitude = df_location_geometry_sorted['geometry.location.lng'][ind]

    url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&intent=browse&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET,
        VERSION,
        latitude,
        longitude,
        9999,
        50,
        category)
    results = json_normalize(requests.get(url).json())
    try:
        if not results.empty:
            response_venues=results['response.venues'][0]
            if response_venues:
                if df_all_hotels.empty:
                    df_all_hotels=pd.DataFrame(response_venues)
                else:
                    df_all_hotels=df_all_hotels.append(response_venues,sort=False)
    except KeyError:
        print('Error')

print(len(df_all_hotels))

In [ ]:
# Remove duplications
df_all_hotels_sorted = df_all_hotels.sort_values(by=['name'], ignore_index=True).reset_index()[['id', 'location', 'name']]
df_all_hotels_sorted.drop_duplicates(subset=['id'],ignore_index=True,inplace=True)
df_all_hotels_sorted.head()

In [ ]:
# Searching for all food venues
category='4d4b7105d754a06374d81259' # Food
df_all_food=pd.DataFrame()

for ind in df_all_hotels_sorted.index:

    location = df_all_hotels_sorted['location'][ind]
    try:
        coord=pd.DataFrame(location['labeledLatLngs'])
        latitude = coord['lat'][0]
        longitude = coord['lng'][0]
    except KeyError: # Some of the locations does not have 'labeledLatLngs', but just JSON string
        try:
            location_json = ast.literal_eval(location)
            latitude = location_json['lat']
            longitude = location_json['lng']
        except ValueError: # Some of those JSON string does not have 'quotes' around the values
            latitude = location['lat']
            longitude = location['lng']

    try:
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&intent=browse&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET,
            VERSION,
            latitude,
            longitude,
            999,
            50,
            category)
        results = json_normalize(requests.get(url).json())

        if not results.empty:
            response_venues=results['response.venues'][0]
            if response_venues:
                if df_all_food.empty:
                    df_all_food=pd.DataFrame(response_venues)
                    df_all_hotels_sorted['food.count'] = 0
                else:
                    df_all_food=df_all_food.append(response_venues,sort=False)
                df_all_hotels_sorted.loc[ind, 'food.count'] = len(response_venues)

    except KeyError:
        print('Error')

print(df_all_food.shape)

In [ ]:
df_all_food_sorted=df_all_food.sort_values(by=['name'],ignore_index=True).reset_index()[['id','location','name']]
df_all_food_sorted.drop_duplicates(subset=['id'],ignore_index=True,inplace=True)
df_all_food_sorted.shape

In [ ]:
df_all_food_sorted.head()

In [ ]:
df_all_food_sorted.to_csv('food_Hotels.csv',index=False)
upload_file_cos(credentials,'food_Hotels.csv','food_Hotels.csv')

In [ ]:
# Searching for all Nightlife venues
category = '4d4b7105d754a06376d81259'  # Nightlife Spot
df_all_Nightlife=pd.DataFrame()

for ind in df_all_hotels_sorted.index:

    location = df_all_hotels_sorted['location'][ind]
    try:
        coord=pd.DataFrame(location['labeledLatLngs'])
        latitude = coord['lat'][0]
        longitude = coord['lng'][0]
    except KeyError: # Some of the locations does not have 'labeledLatLngs', but just JSON string
        try:
            location_json = ast.literal_eval(location)
            latitude = location_json['lat']
            longitude = location_json['lng']
        except ValueError: # Some of those JSON string does not have 'quotes' around the values
            latitude = location['lat']
            longitude = location['lng']

    try:
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&intent=browse&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET,
            VERSION,
            latitude,
            longitude,
            999,
            50,
            category)
        results = json_normalize(requests.get(url).json())

        if not results.empty:
            response_venues=results['response.venues'][0]
            if response_venues:
                if df_all_Nightlife.empty:
                    df_all_Nightlife=pd.DataFrame(response_venues)
                    df_all_hotels_sorted['nightlife.count'] = 0
                else:
                    df_all_Nightlife=df_all_Nightlife.append(response_venues,sort=False)
                df_all_hotels_sorted.loc[ind, 'nightlife.count'] = len(response_venues)

    except KeyError:
        print('Error')

print(df_all_Nightlife.shape)

In [ ]:
df_all_Nightlife_sorted=df_all_Nightlife.sort_values(by=['name'],ignore_index=True).reset_index()[['id','location','name']]
df_all_Nightlife_sorted.drop_duplicates(subset=['id'],ignore_index=True,inplace=True)
df_all_Nightlife_sorted.shape

In [ ]:
df_all_Nightlife_sorted.head()

In [ ]:
df_all_Nightlife_sorted.to_csv('nightlife_Hotels.csv',index=False)
upload_file_cos(credentials,'nightlife_Hotels.csv','nightlife_Hotels.csv')

In [ ]:
df_all_hotels_sorted['food.ratio'] = df_all_hotels_sorted['food.count'] / 50
df_all_hotels_sorted['nightlife.ratio'] = df_all_hotels_sorted['nightlife.count'] / 50
df_all_hotels_sorted.head()

In [ ]:
df_all_hotels_sorted.to_csv('hotels_all.csv',index=False)
upload_file_cos(credentials,'hotels_all.csv','hotels_all.csv')